In [ ]:
import optuna
import pandas as pd
import numpy as np
import time
import os
import sys
from multiprocessing import Pool, cpu_count

# Import algorytmu Tabu Search i narzędzi pomocniczych
from src.algorithms.tabu_move import solve_tsp
from src.utils.tsp_loader import load_tsp_matrix
from src.utils.result_saver import save_experiment_results
from src.utils.run_single_repeat import run_single_repeat

In [ ]:
# Konfiguracja eksperymentu
INSTANCES = [
    'Dane_TSP_48.xlsx',
    'Dane_TSP_76.xlsx',
    'Dane_TSP_127.xlsx'
]

# Liczba prób optymalizacji dla każdej instancji (ile razy Optuna wybierze parametry)
N_TRIALS = 100

# Liczba powtórzeń dla każdego zestawu parametrów
REPEATS = 5

In [ ]:
def objective(trial, distance_matrix):
    """
    Funkcja celu dla Optuny.
    Optuna dobiera parametry, my uruchamiamy algorytm wielokrotnie (REPEATS) 
    i zwracamy wynik (minimalny koszt z powtórzeń).
    """
    # Definiujemy zakresy, z których Optuna może losować wartości.
    max_iter = trial.suggest_int("max_iter", 5_000, 100_000, step=5_000)
    stop_no_improve = trial.suggest_int("stop_no_improve", 500, 10_000, step=500)
    
    tabu_tenure = trial.suggest_int("tabu_tenure", 5, 100, step=5)
    n_neighbors = trial.suggest_int("n_neighbors", 10, 150, step=10)
    
    neighborhood_type = trial.suggest_categorical("neighborhood_type", ["swap", "insert", "two_opt"])
    
    params = {
        "max_iter": max_iter,
        "stop_no_improve": stop_no_improve,
        "tabu_tenure": tabu_tenure,
        "n_neighbors": n_neighbors,
        "neighborhood_type": neighborhood_type
    }
    
    # Uruchomienie Algorytmu Tabu Search wielokrotnie (równolegle)
    with Pool(processes=cpu_count()) as pool:
        parallel_jobs = [
            (solve_tsp, distance_matrix, params) for _ in range(REPEATS)
        ]
        # run_single_repeat zwraca (cost, route, runtime)
        results_parallel = pool.map(run_single_repeat, parallel_jobs)
    
    # Agregacja wyników
    costs = [res[0] for res in results_parallel]
    routes = [res[1] for res in results_parallel]
    runtimes = [res[2] for res in results_parallel]
    
    min_cost = min(costs)
    mean_cost = np.mean(costs)
    mean_runtime = np.mean(runtimes)
    
    # Znalezienie trasy odpowiadającej minimalnemu kosztowi
    best_idx = costs.index(min_cost)
    best_route = routes[best_idx]
    
    # Zapisanie dodatkowych statystyk w atrybutach triala
    route_str = "-".join(map(str, best_route))
    trial.set_user_attr("min_route", route_str)
    trial.set_user_attr("mean_cost", mean_cost)
    trial.set_user_attr("mean_runtime", mean_runtime)
    
    # Optuna minimalizuje wartość zwracaną. 
    # Zwracamy min_cost (najlepszy wynik z serii), aby znaleźć parametry dające szansę na najlepszy wynik.
    # Można by też zwracać mean_cost, jeśli zależy nam na stabilności.
    return min_cost

In [ ]:
experiment_results = []

for instance_file in INSTANCES:
    print(f"\nOptymalizacja dla instancji: {instance_file}")
    
    distance_matrix = load_tsp_matrix(instance_file)
    
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="minimize", sampler=sampler)
    
    # Parametry startowe (warm start)
    study.enqueue_trial({
        "max_iter": 5_000,
        "stop_no_improve": 500,
        "tabu_tenure": 10,
        "n_neighbors": 30,
        "neighborhood_type": "two_opt"
    })

    study.optimize(lambda trial: objective(trial, distance_matrix), n_trials=N_TRIALS)
    
    print(f"Najlepsze parametry dla {instance_file}: {study.best_params}")
    print(f"Najlepszy koszt dla {instance_file}: {study.best_value}")
    
    df_trials = study.trials_dataframe()
    
    for _, row in df_trials.iterrows():
        record = {
            "instance": instance_file,
            "max_iter": row.get("params_max_iter"),
            "tabu_tenure": row.get("params_tabu_tenure"),
            "n_neighbors": row.get("params_n_neighbors"),
            "neighborhood_type": row.get("params_neighborhood_type"),
            "stop_no_improve": row.get("params_stop_no_improve"),
            # Statystyki z powtórzeń
            "min_cost": row.get("value"),
            "mean_cost": row.get("user_attrs_mean_cost"),
            "mean_runtime": row.get("user_attrs_mean_runtime"),
            "min_route": row.get("user_attrs_min_route"),
            "state": row.get("state"),
            "trial_number": row.get("number")
        }
        experiment_results.append(record)


Optymalizacja dla instancji: Dane_TSP_48.xlsx


[I 2025-12-16 11:56:28,169] A new study created in memory with name: no-name-13cf3fd6-90d7-4583-aa72-d05bc106d3b0
[I 2025-12-16 11:56:31,418] Trial 0 finished with value: 9969.0 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 9969.0.
[I 2025-12-16 11:57:12,750] Trial 1 finished with value: 10692.0 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 9969.0.
[I 2025-12-16 11:57:18,407] Trial 2 finished with value: 14946.0 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 9969.0.


Najlepsze parametry dla Dane_TSP_48.xlsx: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_48.xlsx: 9969.0

Optymalizacja dla instancji: Dane_TSP_76.xlsx


[I 2025-12-16 11:57:18,697] A new study created in memory with name: no-name-c2f0400e-bbe5-4e90-8d45-16dd2178e0fb
[I 2025-12-16 11:57:22,668] Trial 0 finished with value: 114585.8027698542 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 114585.8027698542.
[I 2025-12-16 11:58:33,860] Trial 1 finished with value: 122720.11152936376 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 114585.8027698542.
[I 2025-12-16 11:58:37,320] Trial 2 finished with value: 232054.79770888697 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 114585.8027698542.
[I 2025-12-16 11:58:37,450] A new study created in memory with name: no-name-d1321f25-5516-4f27-aacd-a178f67d7cce


Najlepsze parametry dla Dane_TSP_76.xlsx: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_76.xlsx: 114585.8027698542

Optymalizacja dla instancji: Dane_TSP_127.xlsx


[I 2025-12-16 11:58:42,802] Trial 0 finished with value: 136154.2186311292 and parameters: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}. Best is trial 0 with value: 136154.2186311292.
[I 2025-12-16 12:00:45,817] Trial 1 finished with value: 139346.7826995041 and parameters: {'max_iter': 40000, 'stop_no_improve': 10000, 'tabu_tenure': 75, 'n_neighbors': 90, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 136154.2186311292.
[I 2025-12-16 12:00:56,149] Trial 2 finished with value: 254906.28310426278 and parameters: {'max_iter': 90000, 'stop_no_improve': 6500, 'tabu_tenure': 75, 'n_neighbors': 10, 'neighborhood_type': 'swap'}. Best is trial 0 with value: 136154.2186311292.


Najlepsze parametry dla Dane_TSP_127.xlsx: {'max_iter': 5000, 'stop_no_improve': 500, 'tabu_tenure': 10, 'n_neighbors': 30, 'neighborhood_type': 'two_opt'}
Najlepszy koszt dla Dane_TSP_127.xlsx: 136154.2186311292


In [ ]:
if experiment_results:
    df_final = pd.DataFrame(experiment_results)
    
    # Sortowanie wyników: najpierw po instancji, potem po numerze triala
    df_final = df_final.sort_values(by=["instance", "trial_number"])
    
    save_experiment_results(df_final, time_seconds=0, subfolder="TS_Optuna", sort_by_cost=False)
    
    print("\nWyniki zostały zapisane pomyślnie w folderze project/results/TS_Optuna/")
    print(df_final[["instance", "trial_number", "min_cost", "mean_cost", "mean_runtime"]].head())


Podsumowanie (pierwsze 20 wierszy):
         instance  max_iter  tabu_tenure  n_neighbors neighborhood_type  stop_no_improve      min_cost     mean_cost  mean_runtime                                                                                                                                                                                                                                                                                                                                                                                                         min_route    state  trial_number
Dane_TSP_127.xlsx      5000           10           30           two_opt              500 136154.218631 141156.033629      3.890837 108-19-67-73-70-71-68-69-75-76-78-117-84-81-126-82-83-101-102-63-119-96-109-85-87-88-86-110-104-0-98-97-123-95-93-94-46-48-45-103-44-127-107-111-112-49-118-53-47-55-66-65-99-92-89-125-113-64-61-62-91-90-116-59-60-100-58-3-11-9-114-13-51-2-37-27-14-41-36-43-30-80-79-74-72-8-24